In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install xgboost==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
if not os.path.exists('/content/train_data.ftr'):
  
  !pip install kaggle -q
  !mkdir ~/.kaggle 
  !cp kaggle.json ~/.kaggle/

  !rm /content/sample_data -r -q
  !kaggle datasets download -d munumbutt/amexfeather -q



In [ ]:
from zipfile import ZipFile
import os

if not os.path.exists('/content/train_data.ftr'):

  with ZipFile('/content/amexfeather.zip', 'r') as zipObj:
      listOfFileNames = zipObj.namelist()

      list_ext = ['train_data.ftr','test_data.ftr']
    #   list_ext = ['test_data.ftr']


      for f_name in listOfFileNames:
          if f_name in list_ext:
            zipObj.extract(f_name)



In [ ]:
from operator import imod
from flask import Flask, render_template,request,url_for
from werkzeug.utils import secure_filename
import pandas as pd
import numpy as np
import sklearn
import datetime
from time import time
from time import time
import pickle
import shutil
import os
import numpy as np
# from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import datetime
import random
import warnings
warnings.filterwarnings( "ignore" )
import gc


In [ ]:
drive_path = '/content/drive/MyDrive/Applied_AI/casestudy/model_para/'

In [ ]:
# function for doing feature engginering on categorical columns.


def categorical_featute_engg(df):
    # defining all the faetures

    one_hot_feature = ['D_63', 'D_64']
    cat = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_66", "D_68"]
    offset = [2, 1, 2, 2, 3, 2, 3, 2, 2]  # 2 minus minimal value in full train csv

    # Making categoriacl data drame with only categorical features
    df_cat = df[['customer_ID'] + cat + one_hot_feature]

    df_cat[one_hot_feature] = df_cat[one_hot_feature].replace(r'', 'X', regex=True)
    df_cat[one_hot_feature] = df_cat[one_hot_feature].replace(r'-1', 'Y', regex=True)

    df_count = df.groupby('customer_ID')['S_2'].agg(['count'])
    df_count.columns = ['count']

    del df

    for col, s in zip(cat, offset):
        df_cat[col] = np.array(df_cat[col].values) + s
        df_cat[col] = df_cat[col].fillna(1).astype('int8')

    df_cat = df_cat.reset_index(drop=True)


    pipe_one_hot_encoder = pickle.load(open(drive_path+'one_hot_encoder_D_63_D_64.pkl', 'rb'))
    one_hot_ = pickle.load(open(drive_path+'one_hot_encoder_D_63_D_64.pkl_feature_names', 'rb'))

    one_hot_df = pd.DataFrame(pipe_one_hot_encoder.transform(df_cat[one_hot_feature]).toarray().astype('int8'),
                              columns=one_hot_)

    df_cat = df_cat.drop(['D_63', 'D_64'], axis=1)
    df_cat = pd.concat([df_cat, one_hot_df], axis=1)

    del one_hot_df

    df_cat = df_cat.groupby('customer_ID')[list(df_cat.columns)[1:]].agg(['first', 'last', 'nunique'])
    df_cat.columns = ['_'.join(x) for x in df_cat.columns]

    df_cat['customer_ID'] = df_cat.index

    df_cat = df_cat[['customer_ID'] + list(df_cat.columns)[:-1]]
    df_cat = df_cat.reset_index(drop=True)

    for col in df_cat.columns[1:]:
        df_cat[col] = df_cat[col].astype('int8')

    df_cat = df_cat.merge(df_count, on='customer_ID')
    del df_count

    return df_cat

In [ ]:

def numerical_featute(df):
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID', 'S_2']]
    cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
    num_features = [col for col in all_cols if col not in cat_features]

    new_df = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    new_df.columns = ['_'.join(x) for x in new_df.columns]

    for col in list(new_df.columns):
        new_df[col] = new_df[col].astype('float16')

    column_name = list(new_df.columns)
    return new_df

In [ ]:
def feature_standardization(num_df,cat_df):

    cat_colum_std = pickle.load(open(drive_path+'cat_col_std.pkl','rb'))
    cat_colum_not_std = pickle.load(open(drive_path+'cat_colum_not_std.pkl','rb'))

    pipe_num = pickle.load(open(drive_path+'numerical_data_preprocessing_pipeline.pkl','rb'))
    pipe_cat = pickle.load(open(drive_path+'categorical_data_preprocessing_pipeline.pkl','rb'))

    train_num = pipe_num.transform(num_df.iloc[:,1:])

    train_cat_std = pipe_cat.transform(cat_df[cat_colum_std])
    customer = list(cat_df['customer_ID'])
    train_cat_non_std = np.array(cat_df[cat_colum_not_std[1:]])

    final_data = np.concatenate([train_num,train_cat_std,train_cat_non_std],axis=1)

    for i in range(5):

      if i == 0:
        model = xgb.XGBClassifier()
        model.load_model(drive_path+'model/xgb'+f'/model_{i}.xgb')

        predication_prob = model.predict_proba(final_data)
      else:
        model.load_model(drive_path+'model/xgb'+f'/model_{i}.xgb')
        predication_prob += model.predict_proba(final_data)

    predication_prob = predication_prob/5


  # probability of defualting or result == 1

    predication_prob_0 = [x[0] for x in predication_prob]
    predication_prob_1 = [x[1] for x in predication_prob]

    # making df for csv file

    output_df = pd.DataFrame(list(zip(customer,predication_prob_1)),
                             columns=['customer_ID','Prob Default'])
    
    return output_df

In [ ]:
def predict(df):

    columns = list(df.columns)
    if 'target' in columns:
        df = df.drop(['target'],axis=1)

    # categorical Feature Engg
    cat_df = categorical_featute_engg(df)

    # Numerical Feature Engg
    num_df = numerical_featute(df)
    num_df = num_df.reset_index()
    num_df = num_df.replace(np.inf,np.nan)

    result = feature_standardization(num_df, cat_df)

    return result

In [ ]:
# !rm /content/test/ -r

In [ ]:
if not os.path.exists('/content/test/'):
    os.makedirs('/content/test/')
    df = pd.read_feather('test_data.ftr')
    customer = list(df.customer_ID.unique())
    chunk = int(len(df.customer_ID.unique())/3)

    i = [x*chunk for x in range(4)]
    index = [ x for x in zip(i[:-1],i[1:])]

    for i,x in enumerate(index):
            df[df['customer_ID'].isin(customer[x[0]:x[1]])].reset_index(drop=True).to_feather(f'/content/test/test_data_{i}.ftr')
    del df
    _ = gc.collect()

if not os.path.exists('/content/submission/'):
    os.makedirs('/content/submission/')
for i,x in enumerate(os.listdir('/content/test')):
    df = pd.read_feather('/content/test/'+x)
    df = predict(df)
    df.to_csv(f'/content/submission/submission_{i}.csv',index=False)

    

In [ ]:
df = pd.DataFrame()
for x in os.listdir('/content/submission/'):
  df = pd.concat([df,pd.read_csv('/content/submission/submission_0.csv')])

thersold = pickle.load(open(drive_path+'default_thersold.pkl','rb'))
df['predicton'] = [1 if x>=thersold else 0 for x in df['Prob Default']]
df.to_csv('submission.csv',index=False)

In [ ]:
df

,customer_ID,Prob Default,predicton
0,aadf357ccaf0cf38737407fc6a6e2016f28b9e051cc8fb...,0.158306,0
1,aadf40529138693d8c9f8d9e20f38827b16c37a2f05910...,0.002748,0
2,aadf41287b6649a1e67e1dcd0b0c89eae05aef4e12ea74...,0.002516,0
3,aadf5fb584481d0aded76410474e9cedc932285f952ee9...,0.004180,0
4,aadf7b4ea2af97c5daffad68b2ed8178dc9f089c9f4118...,0.995149,1
...,...,...,...
308202,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.013985,0
308203,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.743514,1
308204,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.538288,1
308205,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.228216,0
